# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [34]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,3,1,1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64,3,1,1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(64,64,3,1,1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(64*3*3,128),
            torch.nn.ReLU(),
            torch.nn.Linear(128,10)
        )
        
    def forward(self,x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        res = conv3_out.view(conv3_out.size(0),-1)
        out = self.dense(res)
        return out
    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [35]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    train_acc = 0
    for images,labels in tqdm(train_loader):        
        # TODO:forward + backward + optimize
        images, labels = Variable(images),Variable(labels)
        out = model(images)
        loss = criterion(out,labels)
        train_loss += loss.item()
        predict = torch.max(out,1)[1]
        train_correct = (predict == labels).sum()
        train_acc += train_correct.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Train loss:{:.6f},accuracy:{:.6f}'.format(train_loss/(len(train_dataset)),train_acc/(len(train_dataset))))        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    model.eval()
    test_loss = 0
    test_acc = 0
    for images, labels in tqdm(test_loader):
        images, labels = Variable(images),Variable(labels)
        out = model(images)
        loss = criterion(out, labels)
        test_loss += loss.item()
        predict = torch.max(out,1)[1]
        test_correct = (predict == labels).sum()
        test_acc += test_correct.item()
    print('Test loss:{:.6f},accuracy:{:.6f}'.format(test_loss/(len(test_dataset)),test_acc/(len(test_dataset)))) 

  7%|▋         | 34/468 [00:08<01:42,  4.22it/s]


KeyboardInterrupt: 

#### Q5:
Please print the training and testing accuracy.